In [1]:
%matplotlib inline 
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
dataset = input_data.read_data_sets("data/MNIST/", one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
image_size = 28
image_shape = (image_size, image_size)
image_flat = image_size*image_size
num_class = 10

In [4]:
layer1_filter_size = 5
layer1_input_channel = 1
layer1_output_channel = 16


layer2_filter_size = 5
layer2_output_channel = 36

num_output_fc1 = 128
num_output_fc2 = 10

In [5]:
def get_new_weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [6]:
def get_new_bias(shape):
    return tf.Variable(tf.constant(0.05, shape=shape))

In [7]:
def get_convolutional_layer(input, num_input_channel, num_output_channel, filter_size, max_pooling= True):
    
    shape = [filter_size, filter_size, num_input_channel, num_output_channel]
    
    weight = get_new_weight(shape)
    bias = get_new_bias([num_output_channel])
    
    layer = tf.nn.conv2d(input, filter = weight, strides = [1,1,1,1], padding='SAME')
    
    layer = layer + bias
    
    if max_pooling:
        layer = tf.nn.max_pool(layer, ksize=[1,2,2,1], strides = [1,2,2,1], padding= 'SAME')
        
    layer = tf.nn.relu(layer)
    
    return layer, weight

In [8]:
def get_flat_layer(layer):
    
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1,num_features])
    return layer_flat, num_features

In [9]:
def get_fully_connected_layer(layer_flat, num_input, num_output, relu = True):
    
    weight = get_new_weight([num_input, num_output])
    bias = get_new_bias([num_output])
    
    layer = tf.matmul(layer_flat, weight) + bias
    if relu:
        layer = tf.nn.relu(layer)
        
    return layer, weight

In [10]:
x = tf.placeholder(tf.float32, [None, image_flat])

In [11]:
y = tf.placeholder(tf.float32, [None, num_class])

In [12]:
y_true_class = tf.argmax(y, dimension=1)

In [13]:
x_image = tf.reshape(x, [-1, image_size, image_size, layer1_input_channel])

In [14]:
layer1, weight1 = get_convolutional_layer(x_image, layer1_input_channel, layer1_output_channel, layer1_filter_size, max_pooling=True)

In [15]:
layer1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

In [16]:
weight1

In [17]:
layer2, weight2 =get_convolutional_layer(layer1, layer1_output_channel, layer2_output_channel, layer2_filter_size, max_pooling=True)

In [18]:
layer2

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 36) dtype=float32>

In [19]:
weight2

In [20]:
flat_layer, num_features = get_flat_layer(layer2)

In [21]:
num_features

1764

In [22]:
flat_layer

<tf.Tensor 'Reshape_1:0' shape=(?, 1764) dtype=float32>

In [23]:
layer3, weight3 = get_fully_connected_layer(flat_layer, num_features, num_output_fc1, relu= True)

In [24]:
layer3

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [25]:
weight3

In [26]:
layer4, weight4 = get_fully_connected_layer(layer3, num_output_fc1, num_output_fc2, relu= True)

In [27]:
layer4

<tf.Tensor 'Relu_3:0' shape=(?, 10) dtype=float32>

In [28]:
weight4

In [29]:
y_pred = tf.nn.softmax(layer4)

In [30]:
y_pred_class =  tf.argmax(y_pred, dimension=1)

In [31]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer4,labels=y)

In [32]:
cost = tf.reduce_mean(cross_entropy)

In [33]:
optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)

In [34]:
correct_pred = tf.equal(y_true_class, y_pred_class)

In [35]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [36]:
sess = tf.Session()

In [37]:
sess.run(tf.global_variables_initializer())

In [38]:
BATCH_SIZE = 64

In [39]:
def optimization(training_iterations):
    for iteration in range(training_iterations):
        x_train, y_train = dataset.train.next_batch(BATCH_SIZE)
        feed_dict_train = {x:x_train, y:y_train}
        sess.run(optimizer, feed_dict = feed_dict_train)

In [40]:
dataset.test.cls = np.argmax(dataset.test.labels, axis=1)

In [41]:
def get_accuracy():
    feed_dict_test = {x:dataset.test.images, y:dataset.test.labels, y_true_class:dataset.test.cls}
    print("Accuracy is {0}".format(sess.run(accuracy, feed_dict= feed_dict_test)))


In [42]:
get_accuracy()

Accuracy is 0.06530000269412994


In [43]:
optimization(1)

In [44]:
get_accuracy()

Accuracy is 0.07660000026226044


In [45]:
optimization(99)

In [46]:
get_accuracy()

Accuracy is 0.6162999868392944


In [47]:
optimization(900)

In [ ]:
get_accuracy()

Accuracy is 0.8507000207901001


In [ ]:
optimization(9000)

In [ ]:
get_accuracy()